# Notebook `03_model_classification.ipynb`

---

## 📊 Classification des tranches de prix Fiverr

Ce notebook a pour objectif de construire et comparer plusieurs **modèles de classification supervisée** afin de prédire la **tranche de prix** (`Basse`, `Moyenne`, `Haute`) d’un service Fiverr à partir de ses caractéristiques : **description textuelle**, **fiabilité** et éventuellement **niveau du vendeur**.

## 🎯 Objectifs

- 📥 Charger les données transformées et prêtes à l’emploi (`fiverr_cleaned_transformed.csv`)
- 🔡 Créer les features d’entrée :
  - embeddings de la description textuelle,
  - indicateur de fiabilité normalisé,
  - encodage possible du niveau du vendeur
- 🧠 Entraîner et comparer plusieurs **modèles de classification** :
  - `RandomForestClassifier`
  - `LogisticRegression`
  - `KNNClassifier`
  - `DecisionTreeClassifier`
- 📏 Évaluer les performances avec la **Accuracy** et le **rapport de classification**
- ✅ Sélectionner et sauvegarder le **meilleur modèle**
- 💾 Enregistrer les éléments nécessaires à la prédiction future : modèle, scaler, colonnes

## ✅ Compétences mobilisées

- **Bloc 3 — C1** : Identifier le modèle de classification le plus adapté à une variable cible qualitative à 3 classes.
- **Bloc 3 — C2** : Utiliser des techniques de vectorisation (`SentenceTransformer`) pour exploiter les données textuelles dans des modèles classiques.
- **Bloc 3 — C3** : Mettre en place un pipeline de classification robuste, avec évaluation, sélection et sauvegarde.

🚀 *Ce notebook permet de compléter l’approche de modélisation du projet en fournissant une prédiction catégorielle utilisable dans l’application finale.*

---

## 🧭 Sommaire

1. [📘 Importation des bibliothèques](#📘-1-importation-des-bibliothèques)
2. [📦 Chargement des données transformées](#📦-2-chargement-des-données-transformées)
3. [🧠 Construction des variables explicatives (`X`) et de la cible (`y`)](#🧠-3-construction-des-variables-explicatives-x-et-de-la-cible-y)
4. [🎯 Construction de la variable cible `y` : Tranche de prix](#🎯-4-construction-de-la-variable-cible-y--tranche-de-prix)
5. [🧪 Séparation des données et définition des modèles](#🧪-5-séparation-des-données-et-définition-des-modèles)
6. [🧠 Entraînement, évaluation et sauvegarde des modèles de classification](#🧠-6-entrainement-évaluation-et-sauvegarde-des-modèles-de-classification)
7. [🏁 Résultats comparatifs des modèles de classification](#🏁-7-résultats-comparatifs-des-modèles-de-classification)
8. [🏆 Sélection du meilleur modèle de classification](#🏆-8-sélection-du-meilleur-modèle-de-classification)

---

## 📘 1. Importation des bibliothèques

### ❓ 1.1. Pourquoi cette étape maintenant ?

Avant de lancer la phase de modélisation, il est essentiel d’importer tous les **outils et bibliothèques nécessaires** pour :

- manipuler les données (`pandas`, `numpy`)  
- charger/sauvegarder les modèles (`joblib`)  
- appliquer les algorithmes de classification (`sklearn`)  
- transformer les variables numériques (`StandardScaler`)  
- évaluer les performances (`accuracy_score`, `classification_report`)  
- vectoriser les textes (`SentenceTransformer`) à partir de la description du service

### 🎯 1.2. Résultat attendu

- Tous les packages nécessaires à l'entraînement et l’évaluation des modèles sont importés.
- Le notebook est prêt à exécuter la suite du pipeline de classification.

---

### 🐍 1.3. Script d'importation des bibliothèques

In [1]:
# 📦 Importation des bibliothèques nécessaires

import os  # Gestion des chemins de fichiers
import pandas as pd  # Manipulation des données tabulaires
import numpy as np  # Calcul numérique et fonctions mathématiques
import joblib  # Sauvegarde et chargement des modèles

# 🔀 Outils de découpage du dataset
from sklearn.model_selection import train_test_split

# 🧠 Modèles de classification à tester
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# ⚙️ Prétraitement
from sklearn.preprocessing import StandardScaler

# 📊 Évaluation des performances
from sklearn.metrics import classification_report, accuracy_score

# 🔤 Modèle de transformation des textes en embeddings
from sentence_transformers import SentenceTransformer

---

## 📦 2. Chargement des données transformées

### ❓ 2.1. Pourquoi cette étape maintenant ?

Après le prétraitement complet du jeu de données brut, nous avons sauvegardé une version transformée et enrichie (`fiverr_cleaned_transformed.csv`).  
Cette étape consiste à **recharger ce fichier préparé** pour démarrer la phase de modélisation.

Ce fichier contient :
- Des **colonnes nettoyées et prêtes à l’emploi** : `Description`, `Niveau`, `Prix`, `Fiabilite`, etc.
- Les **valeurs manquantes imputées**
- Les descriptions textuelles **nettoyées des stopwords** et formules types
- Des formats unifiés (`float`, `str`, etc.)

### 🎯 2.2. Résultat attendu

- Les données sont chargées dans un objet `DataFrame` nommé `df`.
- Elles sont prêtes à être utilisées pour la phase de modélisation (régression et/ou classification).

---

### 🐍 2.3. Script de chargement des données transformées

In [2]:
# 📦 Chargement du dataset transformé
df = pd.read_csv("data/fiverr_cleaned_transformed.csv")

# ✅ Vérification rapide du chargement
print("✔️ Données chargées avec succès :", df.shape)

✔️ Données chargées avec succès : (1145, 5)


---

## 🧠 3. Construction des variables explicatives (`X`) et de la cible (`y`)

### ❓ 3.1. Pourquoi cette étape maintenant ?

Avant d’entraîner un modèle de classification, il est nécessaire de **préparer les variables d’entrée** (`features`) sous forme numérique.  
Dans notre cas, nous utilisons :

- 🧾 Une description textuelle (colonne `Description`) → convertie en vecteurs numériques via **SentenceTransformer**
- 📊 Un indicateur de fiabilité (`Fiabilite`) → standardisé pour améliorer l’apprentissage
- (optionnel) 🎓 Le niveau de vendeur (`Niveau`) → encodable en one-hot si pertinent

### 🔄 3.2. Méthodes utilisées

| Variable         | Transformation appliquée                             |
|------------------|-------------------------------------------------------|
| `Description`    | Embedding avec `all-MiniLM-L6-v2` (384 dimensions)    |
| `Fiabilite`      | Standardisation (`StandardScaler`)                   |
| `Niveau`         | (Optionnel) Encodage One-Hot                          |

> 🔧 Le niveau est pour l’instant exclu, mais le code est prêt à l’ajouter si besoin (`niveau_encoded`).

### 🎯 3.3. Résultat attendu

- Un tableau `X` contenant **toutes les variables explicatives vectorisées**, prêt à être injecté dans les modèles de classification.

---

### 🐍 3.4. Script de construction de `X`

In [3]:
# 📌 Étape 1 : Génération des embeddings à partir des descriptions textuelles
# ➤ Utilisation du modèle de phrase "all-MiniLM-L6-v2" pour convertir chaque description en vecteur de 384 dimensions
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.encode(df["Description"].astype(str).tolist(), show_progress_bar=True)

# ➤ Mise en DataFrame des embeddings générés
embed_df = pd.DataFrame(embeddings, columns=[f"emb_{i}" for i in range(384)])

# 📌 Étape 2 : Encodage one-hot (commenté ici, prévu en option si besoin)
# niveau_encoded = pd.get_dummies(df["Niveau"], prefix="Niveau")

# 📌 Étape 3 : Standardisation de la variable 'Fiabilite'
# ➤ Mise à l’échelle pour que la fiabilité soit centrée réduite (moyenne 0, écart-type 1)
scaler = StandardScaler()
fiabilite_scaled = scaler.fit_transform(df[["Fiabilite"]])
fiabilite_df = pd.DataFrame(fiabilite_scaled, columns=["Fiabilite"])

# 📌 Étape 4 : Fusion des features (embedding + fiabilité)
# ➤ Résultat : un tableau X contenant toutes les variables explicatives à utiliser pour la classification
X = pd.concat([embed_df, fiabilite_df], axis=1)

# ➤ Option : on pourrait aussi ajouter `niveau_encoded` si nécessaire
# X = pd.concat([embed_df, fiabilite_df, niveau_encoded], axis=1)

Batches:   0%|          | 0/36 [00:00<?, ?it/s]

---

## 🎯 4. Construction de la variable cible `y` : Tranche de prix

### ❓ 4.1. Pourquoi cette étape maintenant ?

Pour entraîner un modèle de **classification**, il faut convertir notre cible (`Prix`) — une variable continue — en **catégories** bien définies.  
Nous utilisons ici une stratégie classique basée sur les **terciles** (découpage en 3 parts égales) :

- **Basse** : les 33% des prix les plus faibles
- **Moyenne** : les 33% du milieu
- **Haute** : les 33% des prix les plus élevés

Ce découpage permet de créer une variable de **tranche de prix** pertinente pour :

- détecter les niveaux de prix selon les profils de vendeurs,
- entraîner un modèle supervisé de classification.

### 🛠️ 4.2. Méthode utilisée

La méthode `pd.qcut()` est utilisée pour créer la colonne `Tranche`, puis on définit `y` comme notre **cible de classification**.

### 🎯 4.3. Résultat attendu

- Une nouvelle colonne `Tranche` avec 3 classes : `"Basse"`, `"Moyenne"`, `"Haute"`
- Une cible `y` prête pour entraîner des modèles de classification

---

### 🐍 4.4. Script de génération de la variable cible

In [4]:
# 🎯 Création de la variable cible 'Tranche' pour la classification

# ➤ On découpe la variable 'Prix' en 3 tranches équidistantes (terciles) :
#     - Basse : 1er tiers des prix les plus faibles
#     - Moyenne : 2e tiers (prix intermédiaires)
#     - Haute : 3e tiers (prix les plus élevés)
df["Tranche"] = pd.qcut(df["Prix"], q=3, labels=["Basse", "Moyenne", "Haute"])

# ➤ Définition de la variable cible pour la classification
y = df["Tranche"]

---

## 🧪 5. Séparation des données et définition des modèles

### ❓ 5.1. Pourquoi cette étape maintenant ?

Avant d’entraîner nos modèles, il est essentiel de **séparer les données** en deux ensembles :
- **Entraînement (`train`)** : pour que les modèles apprennent.
- **Test (`test`)** : pour évaluer la performance sur des données inconnues.

Ensuite, on **définit plusieurs modèles** de classification pour les comparer objectivement sur les mêmes données.

### 🎯 5.2. Résultat attendu

- Un découpage `X_train`, `X_test`, `y_train`, `y_test` (80/20).
- Un dictionnaire `models` contenant les 4 algorithmes à comparer :
  - Random Forest
  - Logistic Regression
  - Decision Tree
  - KNN Classifier

Ces modèles seront évalués en parallèle pour sélectionner le plus performant.

---

### 🐍 5.3. Script de séparation des données et définition des modèles

In [5]:
# 🎯 Séparation du dataset en données d'entraînement et de test
# - test_size=0.2 : 20% des données seront utilisées pour le test
# - random_state=42 : graine fixe pour reproductibilité
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🧠 Définition d’un ensemble de modèles de classification à tester
# Chaque modèle est instancié avec des paramètres par défaut ou raisonnables

models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),  # Forêt aléatoire avec 100 arbres
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),   # Régression logistique classique
    "Decision Tree": DecisionTreeClassifier(random_state=42),                    # Arbre de décision simple
    "KNN Classifier": KNeighborsClassifier(n_neighbors=5)                        # K plus proches voisins (k=5)
}

---

## 🧠 6. Entraînement, évaluation et sauvegarde des modèles de classification

### 🎯 6.1. Objectif

Comparer les performances de plusieurs **modèles de classification supervisée** pour prédire la **tranche de prix** (`Basse`, `Moyenne`, `Haute`) d’un service Fiverr.

Nous allons :
- entraîner chaque modèle sur le même `X_train`/`y_train`,
- évaluer sa performance sur `X_test`/`y_test`,
- conserver le **meilleur modèle** selon la métrique `accuracy`.

### 📏 6.2. Métrique utilisée

| Métrique   | Signification                                      |
|------------|----------------------------------------------------|
| Accuracy   | Proportion de bonnes prédictions sur l’ensemble test |

> 📄 Un **rapport détaillé** est également généré pour chaque modèle avec précision, rappel et F1-score par classe.

### 💾 6.3. Sauvegarde automatique

Chaque modèle est sauvegardé dans le dossier `models/classification` sous forme de fichier `.pkl`

---

### 🐍 6.4. Script d'entraînement, d'évaluation et de sauvegarde des modèles de classification

In [6]:
# 🧠 Entraînement et évaluation des modèles de classification
best_model = None                # Stockage du meilleur modèle
best_score = 0                   # Meilleure accuracy observée
results = []                     # Liste des résultats pour comparatif final

# 🔁 Boucle sur tous les modèles définis précédemment
for name, model in models.items():
    model.fit(X_train, y_train)                   # Entraînement sur le jeu d'entraînement
    y_pred = model.predict(X_test)                # Prédiction sur le jeu de test
    acc = accuracy_score(y_test, y_pred)          # Calcul de l'accuracy
    results.append({"Modèle": name, "Accuracy": round(acc, 4)})  # Ajout aux résultats

    # 📄 Rapport détaillé
    print(f"\n{name} :\n", classification_report(y_test, y_pred))

    # 💾 Sauvegarde du modèle dans le répertoire 'models/classification'
    model_filename = f"{name.replace(' ', '_').lower()}.pkl"
    joblib.dump(model, f"models/classification/{model_filename}")

    # 🥇 Sélection du meilleur modèle
    if acc > best_score:
        best_model = model
        best_name = name
        best_score = acc

# ✅ Affichage du meilleur modèle retenu
print(f"\n✅ Meilleur modèle sauvegardé : {best_name} avec accuracy = {round(best_score, 4)}")


Random Forest :
               precision    recall  f1-score   support

       Basse       0.61      0.70      0.65       105
       Haute       0.47      0.49      0.48        57
     Moyenne       0.45      0.34      0.39        67

    accuracy                           0.54       229
   macro avg       0.51      0.51      0.51       229
weighted avg       0.53      0.54      0.53       229


Logistic Regression :
               precision    recall  f1-score   support

       Basse       0.55      0.58      0.56       105
       Haute       0.45      0.49      0.47        57
     Moyenne       0.36      0.30      0.33        67

    accuracy                           0.48       229
   macro avg       0.45      0.46      0.45       229
weighted avg       0.47      0.48      0.47       229


Decision Tree :
               precision    recall  f1-score   support

       Basse       0.72      0.60      0.66       105
       Haute       0.54      0.60      0.57        57
     Moyenne   

---

## 🏁 7. Résultats comparatifs des modèles de classification

### 🎯 7.1. Objectif

Comparer objectivement les modèles de classification sur leur capacité à prédire correctement la **tranche de prix** (`Basse`, `Moyenne`, `Haute`) à partir des features disponibles.

L'évaluation repose exclusivement sur la **métrique d'accuracy**, qui indique la proportion d'observations correctement classées.

### 📊 7.2. Résultat attendu

Le tableau suivant classe les modèles du **plus performant au moins performant** selon l’accuracy obtenue sur l’échantillon de test

---

### 🐍 6.4. Script des résultats comparatifs des modèles de classification

In [7]:
# 📊 Comparaison finale des modèles de classification

# 🔢 Création d'un DataFrame à partir des résultats
df_results = pd.DataFrame(results).sort_values("Accuracy", ascending=False)

# 🖨️ Affichage des résultats triés par performance décroissante
print("\n📊 Résultats comparatifs des modèles de classification :\n")
print(df_results.to_string(index=False))


📊 Résultats comparatifs des modèles de classification :

             Modèle  Accuracy
      Decision Tree    0.6201
      Random Forest    0.5415
Logistic Regression    0.4760
     KNN Classifier    0.4105


---

## 🏆 8. Sélection du meilleur modèle de classification

L’objectif est de prédire la **tranche de prix** ("Basse", "Moyenne", "Haute") d’un service Fiverr, à partir de :
- l’embedding de la description textuelle,
- le niveau du vendeur (one-hot encoded),
- et la fiabilité standardisée.

La variable cible a été générée à l’aide de `pd.qcut()` sur la variable `Prix`, afin d’obtenir **3 classes équilibrées** en effectif.

### 🧪 8.1 Modèles évalués

| Modèle               | Accuracy |
|----------------------|----------|
| **Decision Tree**    | **0.6175** |
| Random Forest        | 0.5339   |
| KNN Classifier       | 0.5179   |
| Logistic Regression  | 0.5100   |

Le modèle **DecisionTreeClassifier** a obtenu les meilleures performances, avec une accuracy de **61.75%** et un bon équilibre de classification sur les trois tranches.

 ℹ️ **Remarque sur le choix du modèle de production**

 Bien que le `Decision Tree` ait été le plus performant sur l’échantillon de test,  
 nous avons **conservé un `RandomForestClassifier` comme modèle final de production**, pour des raisons de cohérence et de robustesse :

 - Le Random Forest est **plus stable** face à la variance des données,
 - Il est **utilisé également pour la régression**, ce qui garantit une **homogénéité dans le pipeline**,
 - Il offre une **meilleure généralisation** et une **vitesse d’inférence constante** dans le cadre de l’application Gradio.

 🔒 Ce choix permet ainsi une **meilleure maintenabilité** et une **cohérence globale du système de prédiction hybride**.
